In [1]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

import os
import numpy as np
import shap
import pandas as pd
import random
import matplotlib.pyplot as plt
import xgboost as xgb

from xgboost import plot_importance
import joblib

In [2]:
file_path = 'D:/JuypterNotebook/Github/CasualXGBoost-for-TER/data/ForPCMCI/byPFT/'
csv_list = os.listdir(file_path)
csv_list = [f for f in csv_list if f[-4:] == '.csv']
list_num = len(csv_list)

In [3]:
out_dir = 'D:/JuypterNotebook/Github/CasualXGBoost-for-TER/data/XGBoost/'

In [4]:
weight = pd.read_csv('D:/JuypterNotebook/Github/CasualXGBoost-for-TER/data/Causal_effect_PFT.csv')
weight

,PFT,Air_temperature,Soil_temperature,VPD,SWC,Precipation,SOC_0,SOC_10,SOC_30,LAI,BS,PV,NPV,CI
0,CRO,0.095118,0.059092,-0.038036,0.054661,-0.004960,0.001132,0.001132,0.013933,0.203047,-0.002347,0.114800,-0.061759,-0.188539
1,DBF,0.205556,-0.063205,-0.006156,0.020547,-0.011677,-0.037615,0.006521,0.186071,0.090136,-0.009181,-0.014210,-0.069288,-0.005936
2,EBF,0.077930,0.052589,0.089852,-0.094596,-0.006911,-0.085589,0.054376,0.053910,0.082099,-0.054075,0.077295,-0.084768,-0.550733
3,ENF,0.268388,0.003773,-0.027481,0.013165,-0.012428,-0.018095,0.014095,-0.020663,-0.010045,-0.014592,-0.008571,-0.062217,0.018441
4,GRA,0.042811,0.015401,-0.003516,0.042353,0.233636,0.027398,0.038403,-0.018373,0.364845,-0.023684,0.127569,-0.124530,-0.080207
5,MF.,0.176532,-0.005934,0.075196,0.072577,-0.046755,-0.039582,-0.037269,0.007923,0.102972,0.042912,0.064225,-0.000769,-0.063409
6,SAV,0.080662,0.004045,-0.016766,0.019753,0.162545,-0.051205,0.063061,-0.090611,0.135651,0.046099,0.049577,-0.137008,-0.027794
7,SHR,0.010857,-0.056226,-0.065191,-0.030609,0.265418,-0.034539,0.079018,0.080306,0.103123,0.087029,0.063500,-0.346964,-0.071965


In [5]:
# random_seeds = random.sample(range(1,1000),20)
random_seeds = [837, 929, 321, 536, 710, 505, 213, 802, 671, 428, 839, 901, 683, 826, 277, 176, 399, 105, 776, 792]
print(random_seeds)

[837, 929, 321, 536, 710, 505, 213, 802, 671, 428, 839, 901, 683, 826, 277, 176, 399, 105, 776, 792]


In [32]:
for i in range(list_num):
    
    file_name = file_path + csv_list[i]    
    sheet_file = pd.read_csv(file_name)
    pft = sheet_file.iloc[0,6]
    print(file_name,'\n')
    
    XGB_base_weight = pd.DataFrame()
    XGB_causal_weight = pd.DataFrame()
    
    for j in range(len(random_seeds)):
        
        order = j+1
        
        sub_train = sheet_file.groupby('siteID').apply(pd.DataFrame.sample,
                                                       frac = 0.8, random_state = random_seeds[j]).reset_index(level = 'siteID',drop = True)
        sub_notra = sheet_file.drop(index=sub_train.index)
        
        x_train = sub_train.iloc[:,12:]
        y_train = sub_train.iloc[:,11]
        
        x_test, x_val, y_test, y_val = train_test_split(sub_notra.iloc[:,12:], sub_notra.iloc[:,11],
                                                        test_size=0.5,random_state=random_seeds[j])
        
        valid_set = [(x_train, y_train),(x_val, y_val)]
        
        model_base = xgb.XGBRegressor(n_estimators = 10000, early_stopping_rounds = 30,device = 'cpu',
                                      learning_rate = 0.05, max_depth = 5, min_child_weight = 3,
                                      gamma = 0.4, subsample = 0.8,
                                      colsample_bynode = 0.8, colsample_bytree = 1, colsample_bylevel = 0.6,
                                      random_state = 666
                                     )

        model_causal = xgb.XGBRegressor(n_estimators = 10000, early_stopping_rounds = 30,device = 'cpu',
                                        learning_rate = 0.05, max_depth = 5, min_child_weight = 3,
                                        gamma = 0.4, subsample = 0.8,
                                        colsample_bynode = 0.8, colsample_bytree = 1, colsample_bylevel = 0.6,
                                        random_state = 666
                                       )
        
        XGB_Base = model_base.fit(x_train, y_train, eval_set = valid_set, verbose = False)
        XGB_Causal = model_causal.fit(x_train, y_train, eval_set = valid_set, verbose = False,
                                      feature_weights = abs(weight.iloc[i,1:].to_numpy()) )
        
        # weight save        
        base_wight = pd.DataFrame([XGB_Base.get_booster().get_score(importance_type= 'weight')])
        causal_wight = pd.DataFrame([XGB_Causal.get_booster().get_score(importance_type= 'weight')])
        
        XGB_base_weight = pd.concat([XGB_base_weight, base_wight])
        XGB_causal_weight = pd.concat([XGB_causal_weight, causal_wight])
        
        # save learning curve
        eval_base = XGB_Base.evals_result()
        eval_causal = XGB_Causal.evals_result()
        
        loss_base = pd.DataFrame([eval_base['validation_0']['rmse'],eval_base['validation_1']['rmse']]).T
        loss_causal = pd.DataFrame([eval_causal['validation_0']['rmse'],eval_causal['validation_1']['rmse']]).T
        
        loss_result = pd.concat([loss_base,loss_causal],axis=1)
        loss_result.columns = ['base_train','base_valid','causal_train','causal_valid']
        
        loss_result.to_csv(out_dir+'Loss_curve/' + pft +'_loss_curve_'+ "%02d" %order +'_Seed_' + "%03d" %random_seeds[j] + '.csv',index = False )
        
        # save test predictions
        TER_REF = y_test.reset_index(drop=True)
        base_pred = pd.DataFrame(XGB_Base.predict(x_test)) 
        causal_pred = pd.DataFrame(XGB_Causal.predict(x_test))
        
        predicts = pd.concat([TER_REF,base_pred,causal_pred],axis=1)
        predicts.columns = ['RECO_flux','RECO_base','RECO_causal']
        
        predicts.to_csv(out_dir+'Prediction/' + pft +'_Test_prediction_'+ "%02d" %order +'_Seed_' + "%03d" %random_seeds[j] + '.csv',index = False )
        
        # save model
        joblib.dump(XGB_Base, out_dir + 'Model/Baseline_'+ pft + "_%02d" %order + '.joblib')
        joblib.dump(XGB_Causal,  out_dir + 'Model/Causal_'+ pft + "_%02d" %order + '.joblib')
    
    # export xgb weight
    XGB_base_weight.to_csv(out_dir + 'Weight/Baseline_'+ pft +'.csv',index = False)
    XGB_causal_weight.to_csv(out_dir + 'Weight/Causal_'+ pft  +'.csv',index = False)

D:/小论文_Paper(doing)/Paper_09_site scale causality guide XGBoost model for TER/Process/ForPCMCI/byPFT/sub_noNA_CRO.csv 

D:/小论文_Paper(doing)/Paper_09_site scale causality guide XGBoost model for TER/Process/ForPCMCI/byPFT/sub_noNA_DBF.csv 

D:/小论文_Paper(doing)/Paper_09_site scale causality guide XGBoost model for TER/Process/ForPCMCI/byPFT/sub_noNA_EBF.csv 

D:/小论文_Paper(doing)/Paper_09_site scale causality guide XGBoost model for TER/Process/ForPCMCI/byPFT/sub_noNA_ENF.csv 

D:/小论文_Paper(doing)/Paper_09_site scale causality guide XGBoost model for TER/Process/ForPCMCI/byPFT/sub_noNA_GRA.csv 

D:/小论文_Paper(doing)/Paper_09_site scale causality guide XGBoost model for TER/Process/ForPCMCI/byPFT/sub_noNA_MF.csv 

D:/小论文_Paper(doing)/Paper_09_site scale causality guide XGBoost model for TER/Process/ForPCMCI/byPFT/sub_noNA_SAV.csv 

D:/小论文_Paper(doing)/Paper_09_site scale causality guide XGBoost model for TER/Process/ForPCMCI/byPFT/sub_noNA_SHR.csv 



## leave one validation

In [8]:
for i in range(list_num):
    
    file_name = file_path + csv_list[i]    
    sheet_file = pd.read_csv(file_name)
    pft = sheet_file.iloc[0,6]
    print(file_name,'\n')
    
    site = np.unique(sheet_file['siteID'])
    
    for j in range(len(site)):
        
        siteID = site[j]
        test = sheet_file[sheet_file['siteID'] == siteID]
        train = sheet_file.drop(index = test.index)
        
        x_test = test.iloc[:,12:]
        y_test = test.iloc[:,11]
        
        predicts_site = y_test.reset_index(drop=True)
        
        for k in range(20):
            
            order = k+1
            
            x_train, x_val, y_train, y_val = train_test_split(train.iloc[:,12:], train.iloc[:,11],
                                                              test_size=0.8, random_state = random_seeds[k])
            
            valid_set = [(x_train, y_train),(x_val, y_val)]
            
            model_base = xgb.XGBRegressor(n_estimators = 10000, early_stopping_rounds = 30,device = 'cpu',
                                          learning_rate = 0.05, max_depth = 5, min_child_weight = 3,
                                          gamma = 0.4, subsample = 0.8,
                                          colsample_bynode = 0.8, colsample_bytree = 1, colsample_bylevel = 0.6,
                                          random_state = 666
                                         )

            model_causal = xgb.XGBRegressor(n_estimators = 10000, early_stopping_rounds = 30,device = 'cpu',
                                            learning_rate = 0.05, max_depth = 5, min_child_weight = 3,
                                            gamma = 0.4, subsample = 0.8,
                                            colsample_bynode = 0.8, colsample_bytree = 1, colsample_bylevel = 0.6,
                                            random_state = 666
                                           )
            
            XGB_Base = model_base.fit(x_train, y_train, eval_set = valid_set, verbose = False)
            XGB_Causal = model_causal.fit(x_train, y_train, eval_set = valid_set, verbose = False,
                                          feature_weights = abs(weight.iloc[i,1:].to_numpy()) )
            
            # save prediction for each site
            base_pred = pd.DataFrame(XGB_Base.predict(x_test)) 
            causal_pred = pd.DataFrame(XGB_Causal.predict(x_test))
            
            pred_times = pd.concat([base_pred,causal_pred],axis=1)
            pred_times.columns = ["RECO_base_"+"%02d"%order,"RECO_causal_"+"%02d"%order]
            
            predicts_site = pd.concat([predicts_site,pred_times], axis=1)
        
        # export 20-round predictions for each site
        predicts_site.to_csv(out_dir + 'LeaveOneCrossValidation/LeaveOne_' + pft +"_SiteID_%03d"%siteID + '.csv',index = False )

D:/小论文_Paper(doing)/Paper_09_site scale causality guide XGBoost model for TER/Process/ForPCMCI/byPFT/sub_noNA_CRO.csv 

D:/小论文_Paper(doing)/Paper_09_site scale causality guide XGBoost model for TER/Process/ForPCMCI/byPFT/sub_noNA_DBF.csv 

D:/小论文_Paper(doing)/Paper_09_site scale causality guide XGBoost model for TER/Process/ForPCMCI/byPFT/sub_noNA_EBF.csv 

D:/小论文_Paper(doing)/Paper_09_site scale causality guide XGBoost model for TER/Process/ForPCMCI/byPFT/sub_noNA_ENF.csv 

D:/小论文_Paper(doing)/Paper_09_site scale causality guide XGBoost model for TER/Process/ForPCMCI/byPFT/sub_noNA_GRA.csv 

D:/小论文_Paper(doing)/Paper_09_site scale causality guide XGBoost model for TER/Process/ForPCMCI/byPFT/sub_noNA_MF.csv 

D:/小论文_Paper(doing)/Paper_09_site scale causality guide XGBoost model for TER/Process/ForPCMCI/byPFT/sub_noNA_SAV.csv 

D:/小论文_Paper(doing)/Paper_09_site scale causality guide XGBoost model for TER/Process/ForPCMCI/byPFT/sub_noNA_SHR.csv 

